In [ ]:
#あらかじめ以下のソフトウェアをインストールしておく．
!pip install Fasttext
!pip install ripser
!pip install gensim
!pip install persim

#モジュール，ライブラリのインポート
import download_vec
import calc_homology
import bars_statistics
import random
import numpy as np
from persim import plot_diagrams
import os
import matplotlib.pyplot as plt

#パスの移動
os.chdir('../data')
path = os.getcwd()
print(f"current path : {path}")

In [ ]:
#印欧語族に含まれる30言語
#hy:アルメニア語，lt:ルクセンブルク語，be:ベラルーシ語，ru:ロシア語，uk:ウクライナ語，bg:ブルガリア語，
#hr:クロアチア語，sr:セルビア語，sl:スロベニア語，cs:チェコ語，sk:スロバキア語，pl:ポーランド語，
#cy:ウェールズ語，no:ノルウェー語，da:デンマーク語，sv:スウェーデン語，en:英語，de:ドイツ語，
#nl:オランダ語，el:ギリシア語，bn:ベンガル語，hi:ヒンディー語，ur:ウルドゥー語，ro:ルーマニア語，
#it:イタリア語，fr:フランス語，ca:カタルーニャ語，es:スペイン語，gl:ガリシア語，pt:ポルトガル語
lang_list = ['hy','lt','be','ru','uk','bg','hr','sr','sl','cs','sk','pl','cy','no','da','sv','en','de','nl','el','bn','hi','ur','ro','it','fr','ca','es','gl','pt']

for lang in lang_list:
    filename = f'{lang}.vec.gz'
    filepath = os.path.join(path, filename)

    if not os.path.exists(filepath):
        download_vec.download_fasttext_vec(lang, path)
        print(f"{filename} の保存が完了しました。\n")
    else:
        print(f"{filename} はすでにダウンロード済みです。\n")
        continue

In [ ]:
#num_1単語からnum_2単語をランダムに選び計算するプロセスをiter回繰り返す
num_1 = 200000
num_2 = 10000
iter = 10
random.seed(42)

for lang in lang_list:
    # 点群データ（ベクトル）、単語データの読み込み
    pointcloud, words = calc_homology.vec_frequent(lang, num_1)
    for i in range(iter):
        print(f"{i}回目の計算")
        #ランダムにnum_1個からnum_2個選んでサブサンプリング(重複あり)
        idx = random.choices(range(num_1), k=num_2)
        sample = pointcloud[idx]

        for metric in ['euclidian','cosine']:
            #パーシステント図の計算
            diagram = calc_homology.vietris_rips(sample, metric, lang)

            # 保存(np.load(f"diagram_{lang}_iter{i}.npz")で読み込みできる)
            np.savez(
                f"{lang}_{metric}_iter{i}.npz",
                *diagram
            )

In [ ]:
#基本統計量のプロット
for col in ['birth','death','persistence']:
    for sta in ['mean','median','stdev','IQR','range','10%','25%','75%','90%']:
        bars_statistics.plot_point(1, lang_list, col, sta, x=['cosine',1], y=['euclidian',1], annotate=True)